In [0]:
import requests
import json
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, LongType, TimestampType, BinaryType, IntegerType, DateType
from datetime import datetime
import time
import zipfile
import os
import pandas as pd 

In [0]:
dbutils.widgets.text("fab", "fab") 
dbutils.widgets.text("period", "period") 
dbutils.widgets.text("table", "table") 

fab =  dbutils.widgets.get("fab")
period = dbutils.widgets.get("period") 
table =  dbutils.widgets.get("table")

# fab = "D21"
# table = "vw_D21_3S_TRANS_OIL_AZURE_MONTH"
# period "M"

In [0]:
file_info_schema = StructType([
    StructField('DATE', StringType(), True),
    StructField('STATION', StringType(), True),
    StructField('G', StringType(), True),
    StructField('QTY', StringType(), True),
    StructField('ETL_DATE', StringType(), True)
  
])

convert_udf = udf(lambda x: str(time.strftime("%Y-%m-%dT06:%M:%SZ", time.strptime(str(x),"%Y%m%d"))),StringType())  # python udf

try:
    raw_df = (spark.read.format('csv')
          .option('header',True)
          .schema(file_info_schema)
          .load(f'/mnt/source/incremental/{table}.csv')
          .withColumn('timestamp',convert_udf(F.col('DATE')))
          .withColumnRenamed('QTY','value')
          .select('timestamp','STATION','G','value')
          .repartition('station')
          .write
          .format('delta')
          .mode('append')
          .save(f'/mnt/deltalake/{table}')
             )
except Exception as e:
    print(e)

In [0]:
#pandas df
raw_df = spark.read.format('delta').load(f'/mnt/deltalake/{table}')
raw_df_pandas = raw_df.toPandas()

station_list = list(raw_df_pandas['STATION'].unique())
for name in station_list:
    print(name + ' ziptime :' + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    G_list = list(raw_df_pandas[raw_df_pandas['STATION'] == name]['G'].unique())
    
    dbutils.fs.mkdirs(f'file:/databricks/driver/{fab}/{period}/{name}')
    for g in G_list:
        select_name = raw_df_pandas['STATION'] == name
        select_g = raw_df_pandas['G'] == g
        raw_df_pandas[(select_name & select_g)][['timestamp','value']].sort_values(by=['timestamp']).reset_index(drop=True).to_csv(f'/databricks/driver/{fab}/{period}/{name}/{g}.csv',index=False)  
          
    z = zipfile.ZipFile(f'/databricks/driver/{fab}/{period}/{name}.zip', "w", zipfile.ZIP_DEFLATED)
    for f in os.listdir(f'/databricks/driver/{fab}/{period}/{name}'):
        if f.endswith("csv"):
            z.write(f'/databricks/driver/{fab}/{period}/{name}/{f}' , f )
    z.close()
    dbutils.fs.mv(f'file:/databricks/driver/{fab}/{period}/{name}.zip', f'/mnt/zipfile/{fab}/{period}/{name}.zip')
    dbutils.fs.rm(f'file:/databricks/driver/{fab}/{period}/{name}', True)

a582e74569f9aecc1be3 ziptime :2022-04-26 09:01:08
6a951514fc1b21aeb4e7 ziptime :2022-04-26 09:01:12
4b88ddf4d88af3f823f4 ziptime :2022-04-26 09:01:15
0b08c67ca54995f4451d ziptime :2022-04-26 09:01:19
2220b25b94a8f6595361 ziptime :2022-04-26 09:01:22
87f6a3f3e81eb13df05c ziptime :2022-04-26 09:01:25
c1b1eeb67136d78d732a ziptime :2022-04-26 09:01:27
843d4d0d1735d35e67b5 ziptime :2022-04-26 09:01:30
dc4fc3956dd2c3f6fdfe ziptime :2022-04-26 09:01:33
560d5d0c26d0a9443286 ziptime :2022-04-26 09:01:36
2152e607b099210ab51c ziptime :2022-04-26 09:01:38
07b8ada8541584b38731 ziptime :2022-04-26 09:01:41
6bb5101dd5495979eab4 ziptime :2022-04-26 09:01:44
be8cd02a1b08dc20b395 ziptime :2022-04-26 09:01:47
4993b3eab544c8402f57 ziptime :2022-04-26 09:01:50
7d484d0f5b4953111176 ziptime :2022-04-26 09:01:52
e2bc1049623b81e0e148 ziptime :2022-04-26 09:01:56
6bb203842619cf126123 ziptime :2022-04-26 09:01:58
973c48a18b5d842c277c ziptime :2022-04-26 09:02:01
8bfeee5cf56f731a7e28 ziptime :2022-04-26 09:02:04
87a2b24c36812ca0447a ziptime :2022-04-26 09:02:07
2634cedbfc6b9665ec67 ziptime :2022-04-26 09:02:10
8a820f6b379c441c4301 ziptime :2022-04-26 09:02:12
2c61cef816e8e369a706 ziptime :2022-04-26 09:02:15
6e3c7f8221b117d16999 ziptime :2022-04-26 09:02:18
f76bd13ede09da9a75af ziptime :2022-04-26 09:02:21
8ff0dbd0b9a616496a99 ziptime :2022-04-26 09:02:24
6926a9ad76dcb7750bc7 ziptime :2022-04-26 09:02:27
2465da018b6d28f185d0 ziptime :2022-04-26 09:02:30
c7ebe1f6716ef6ad3c21 ziptime :2022-04-26 09:02:33
20cc23709cc1ee2e354a ziptime :2022-04-26 09:02:36
8bbdaaf09ef12dc5e59a ziptime :2022-04-26 09:02:38
f2958352f8c19453402e ziptime :2022-04-26 09:02:41
0a0da4158168d1679b8a ziptime :2022-04-26 09:02:44
9455781fbb8c620854bc ziptime :2022-04-26 09:02:47
4f33471831c8c3b9eed9 ziptime :2022-04-26 09:02:49
73a0931ae56caefbd7e4 ziptime :2022-04-26 09:02:52
a3f57bcf20363157536a ziptime :2022-04-26 09:02:55
7dbb8f07c87526d1b4bb ziptime :2022-04-26 09:02:57
7894a5e2b64e2b635d03 ziptime :2022-04-26 09:03:00
fa926542c6e1b484106e ziptime :2022-04-26 09:03:03
4432b7fbf8cef90bf141 ziptime :2022-04-26 09:03:06
f47b8c6f5534f1771f38 ziptime :2022-04-26 09:03:09
65c3228790ec9f741290 ziptime :2022-04-26 09:03:12
15787192a2cf526deaf8 ziptime :2022-04-26 09:03:15
a645cc7115535f737fd6 ziptime :2022-04-26 09:03:18
f1ded164dd22c40b8281 ziptime :2022-04-26 09:03:21
0fb6ba15172179e2f690 ziptime :2022-04-26 09:03:23
c7d1438af931170d0a44 ziptime :2022-04-26 09:03:26
1b06cfe8d00e1293ef0c ziptime :2022-04-26 09:03:29
7c13eb09b8bc224b7422 ziptime :2022-04-26 09:03:32
6dcbb61d4eb1762c312f ziptime :2022-04-26 09:03:34
d31ef357a743cacadf67 ziptime :2022-04-26 09:03:37
cebfef740916bde50b34 ziptime :2022-04-26 09:03:40
2a8d9b0331604293642e ziptime :2022-04-26 09:03:43
021512a0fa0166f54dd7 ziptime :2022-04-26 09:03:45
a20823a1799e55d76dae ziptime :2022-04-26 09:03:48
a57ed9bd0d85b706f1e3 ziptime :2022-04-26 09:03:51
dd990a83725dbf0c827c ziptime :2022-04-26 09:03:54
57f5212485819886222e ziptime :2022-04-26 09:03:56
c037ff7f2f90fcc386a7 ziptime :2022-04-26 09:03:59
01a37bc3b6a23ccdaf5b ziptime :2022-04-26 09:04:02
c93d1426f8f158e914b8 ziptime :2022-04-26 09:04:05
3616dfc6526e3349ba1c ziptime :2022-04-26 09:04:08
05f098ac2faab7324ae5 ziptime :2022-04-26 09:04:12
7b8aecf05d619705c674 ziptime :2022-04-26 09:04:14
63bdbd3a26d05c7921cb ziptime :2022-04-26 09:04:17
96fbd704b2af6007a832 ziptime :2022-04-26 09:04:20
dc9c6d3cbd99c7eacc7f ziptime :2022-04-26 09:04:23
c4f097cf28b92520f54a ziptime :2022-04-26 09:04:26
5ca735935a28ac9b40c6 ziptime :2022-04-26 09:04:29
7d5d2d1262b5cf8f0c2a ziptime :2022-04-26 09:04:32
26486f01dd1e1ba30512 ziptime :2022-04-26 09:04:34
c522054e6959b08c249e ziptime :2022-04-26 09:04:37
9526122690d25f6d6d86 ziptime :2022-04-26 09:04:40

In [0]:
# spark df
# list_station = df.select('STATION').drop_duplicates().rdd.collect()
# for row in list_station:
#     print(row.STATION + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
#     df_station = df.filter(F.col('STATION') == row.STATION )
#     list_g = df_station.select('G').drop_duplicates().rdd.collect()
    
#     dbutils.fs.mkdirs(f'file:/databricks/driver/{row.STATION}')
#     for rowg in list_g:
#         df_station_g = (df_station.filter(F.col('G') == rowg.G )
#                        .select('timestamp','value')
#                        )
#         df_station_g_pandas = df_station_g.toPandas()
#         df_station_g_pandas.to_csv(f'/databricks/driver/{row.STATION}/{rowg.G}.csv',index=False)
    
#     z = zipfile.ZipFile(f'/databricks/driver/{row.STATION}.zip', "w", zipfile.ZIP_DEFLATED)
#     for f in os.listdir(f'/databricks/driver/{row.STATION}'):
#         if f.endswith("csv"):
#             z.write(f'/databricks/driver/{row.STATION}/{f}',f)
#     z.close()
#     dbutils.fs.mv(f'file:/databricks/driver/{row.STATION}.zip', f'/mnt/zipfile/{row.STATION}.zip')
#     dbutils.fs.rm(f'file:/databricks/driver/{row.STATION}', True)